In [2]:
import pandas as pd
df = pd.read_csv('email_table.csv')
email = pd.read_csv('email_opened_table.csv')
link  = pd.read_csv('link_clicked_table.csv')
df.head()

,email_id,email_text,email_version,hour,weekday,user_country,user_past_purchases
0,85120,short_email,personalized,2,Sunday,US,5
1,966622,long_email,personalized,12,Sunday,UK,2
2,777221,long_email,personalized,11,Wednesday,US,2
3,493711,short_email,generic,6,Monday,UK,1
4,106887,long_email,generic,14,Monday,US,6


In [3]:
df.describe()

,email_id,hour,user_past_purchases
count,100000.000000,100000.000000,100000.00000
mean,498690.196160,9.059300,3.87845
std,289230.727534,4.439637,3.19611
min,8.000000,1.000000,0.00000
25%,246708.250000,6.000000,1.00000
50%,498447.000000,9.000000,3.00000
75%,749942.750000,12.000000,6.00000
max,999998.000000,24.000000,22.00000


In [8]:
df.user_country.unique()

array(['US', 'UK', 'FR', 'ES'], dtype=object)

In [15]:
l=[]
for i in range (email.shape[0]):
    l.append(1)
email['open_email'] = l
email.head()

,email_id,open_email
0,284534,1
1,609056,1
2,220820,1
3,905936,1
4,164034,1


In [19]:
temp = df.join(email.set_index('email_id'), on='email_id', how='left')
temp = temp.fillna(0)
temp.head()

,email_id,email_text,email_version,hour,weekday,user_country,user_past_purchases,open_email
0,85120,short_email,personalized,2,Sunday,US,5,0.0
1,966622,long_email,personalized,12,Sunday,UK,2,1.0
2,777221,long_email,personalized,11,Wednesday,US,2,0.0
3,493711,short_email,generic,6,Monday,UK,1,0.0
4,106887,long_email,generic,14,Monday,US,6,0.0


In [21]:
l=[]
for i in range (link.shape[0]):
    l.append(1)
link['open_link'] = l
result = temp.join(link.set_index('email_id'), on='email_id', how='left')
result = result.fillna(0)
result.head()

,email_id,email_text,email_version,hour,weekday,user_country,user_past_purchases,open_email,open_link
0,85120,short_email,personalized,2,Sunday,US,5,0.0,0.0
1,966622,long_email,personalized,12,Sunday,UK,2,1.0,1.0
2,777221,long_email,personalized,11,Wednesday,US,2,0.0,0.0
3,493711,short_email,generic,6,Monday,UK,1,0.0,0.0
4,106887,long_email,generic,14,Monday,US,6,0.0,0.0


In [22]:
result[result['open_link']==1].shape

(2119, 9)

In [23]:
result[result['open_email']==1].shape

(10345, 9)

In [27]:
d={'Monday':1, 'Tuesday':2, 'Wednesday': 3, 'Thursday':4, 'Friday':5, 'Saturday':6, 'Sunday':7}
for index, row in result.iterrows():
    result.at[index, 'weekday'] = d[row['weekday']]

In [28]:
result.head()

,email_id,email_text,email_version,hour,weekday,user_country,user_past_purchases,open_email,open_link
0,85120,short_email,personalized,2,7,US,5,0.0,0.0
1,966622,long_email,personalized,12,7,UK,2,1.0,1.0
2,777221,long_email,personalized,11,3,US,2,0.0,0.0
3,493711,short_email,generic,6,1,UK,1,0.0,0.0
4,106887,long_email,generic,14,1,US,6,0.0,0.0


In [34]:
for index, row in result.iterrows():
    if row['weekday']>=1 and row['weekday']<=5:
        result.at[index, 'day'] = 'weekday'
    else:
        result.at[index, 'day'] = 'weekend'
    if row['hour']>=8 and row['hour']<=18:
        result.at[index, 'daytime'] = 'working'
    elif row['hour']>18 and row['hour']<22:
        result.at[index, 'daytime'] = 'evening'
    elif row['hour']>=22 and row['hour']<=24:
        result.at[index, 'daytime'] = 'night' 
    else:
        result.at[index, 'daytime'] = 'sleeping'

In [35]:
result.head()

,email_id,email_text,email_version,hour,weekday,user_country,user_past_purchases,open_email,open_link,day,daytime
0,85120,short_email,personalized,2,7,US,5,0.0,0.0,weekend,sleeping
1,966622,long_email,personalized,12,7,UK,2,1.0,1.0,weekend,working
2,777221,long_email,personalized,11,3,US,2,0.0,0.0,weekday,working
3,493711,short_email,generic,6,1,UK,1,0.0,0.0,weekday,sleeping
4,106887,long_email,generic,14,1,US,6,0.0,0.0,weekday,working


In [32]:
result.to_csv('first_version.csv')

In [67]:
import h2o
from h2o.estimators import H2ORandomForestEstimator
from h2o.model import H2OBinomialModel
h2o.init()
from __future__ import print_function
import sklearn
import sklearn.datasets
import sklearn.ensemble
from sklearn.metrics import confusion_matrix
from sklearn.cross_validation import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import numpy as np
class h2o_predict_proba_wrapper:
    # drf is the h2o distributed random forest object, the column_names is the
    # labels of the X values
    def __init__(self,model,column_names):
            
            self.model = model
            self.column_names = column_names
 
    def predict_proba(self,this_array):        
        # If we have just 1 row of data we need to reshape it
        shape_tuple = np.shape(this_array)        
        if len(shape_tuple) == 1:
            this_array = this_array.reshape(1, -1)
            
        # We convert the numpy array that Lime sends to a pandas dataframe and
        # convert the pandas dataframe to an h2o frame
        self.pandas_df = pd.DataFrame(data = this_array,columns = self.column_names)
        self.h2o_df = h2o.H2OFrame(self.pandas_df)
        
        # Predict with the h2o drf
        self.predictions = self.model.predict(self.h2o_df).as_data_frame()
        # the first column is the class labels, the rest are probabilities for
        # each class
        self.predictions = self.predictions.iloc[:,1:].as_matrix()
        return self.predictions
l = [1,2,3,4,5,6,8,9,10]
data = linkdf.iloc[:,l]

feature_names = ['email_text','email_version','hour','weekday','user_country','user_past_purchases','day','daytime'] 

class_names = 'open_link'

labels = np.array(data.iloc[:,6])
le= sklearn.preprocessing.LabelEncoder()
le.fit(labels)
labels = le.transform(labels)
class_names = le.classes_

datacolumn=[0,1,2,3,4,5,7,8]
data = np.array(data.iloc[:,datacolumn]) # no label no name for column
categorical_features = [0,1,4,6,7]
categorical_names = ['email_text','email_version','user_country','day','daytime' ]
categorical_names = {}
for feature in categorical_features:
    le = sklearn.preprocessing.LabelEncoder()
    le.fit(data[:, feature])
    data[:, feature] = le.transform(data[:, feature])
    categorical_names[feature] = le.classes_
data = data.astype(float)

encoder = sklearn.preprocessing.OneHotEncoder(categorical_features=categorical_features)



train, test, labels_train, labels_test = train_test_split(data, labels, train_size=0.70)


train_h2o_df = h2o.H2OFrame(train)
train_h2o_df.set_names(feature_names)
train_h2o_df['open_link'] = h2o.H2OFrame(labels_train).asfactor()


test_h2o_df = h2o.H2OFrame(test)
test_h2o_df.set_names(feature_names)
test_h2o_df['open_link'] = h2o.H2OFrame(labels_test).asfactor()



for feature in categorical_features:
    train_h2o_df[feature] = train_h2o_df[feature].asfactor()
    test_h2o_df[feature] = test_h2o_df[feature].asfactor()
    

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,2 days 2 hours 56 mins
H2O cluster timezone:,America/New_York
H2O data parsing timezone:,UTC
H2O cluster version:,3.18.0.11
H2O cluster version age:,3 months and 3 days
H2O cluster name:,H2O_from_python_yiyuanfeng_citfvz
H2O cluster total nodes:,1
H2O cluster free memory:,1.633 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"locked, healthy"


Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [65]:
email_drf = H2ORandomForestEstimator(
        model_id="email",
        ntrees=100,
        #stopping_rounds=2,
        score_each_iteration=True,
        #seed=1000000,
        nfolds=4,
        balance_classes=False,
        histogram_type="AUTO")

email_drf.train(x=feature_names,
         y='open_email',
         training_frame=train_h2o_df)

drf Model Build progress: |███████████████████████████████████████████████| 100%


In [66]:
email_drf.coef_norm

Model Details
H2ORandomForestEstimator :  Distributed Random Forest
Model Key:  email


ModelMetricsBinomial: drf
** Reported on train data. **

MSE: 0.1790953850827839
RMSE: 0.4231966269747242
LogLoss: 0.5383209825019036
Mean Per-Class Error: 0.38332713439229416
AUC: 0.6570559186195986
Gini: 0.3141118372391971
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.19404620979367895: 


,0,1,Error,Rate
0,10092.0,11657.0,0.536,(11657.0/21749.0)
1,1736.0,5480.0,0.2406,(1736.0/7216.0)
Total,11828.0,17137.0,0.4624,(13393.0/28965.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.1940462,0.4500472,268.0
max f2,0.0815488,0.6394149,353.0
max f0point5,0.2989970,0.3858982,191.0
max accuracy,0.6765098,0.7512860,13.0
max precision,0.7536412,0.8,2.0
max recall,0.0066992,1.0,399.0
max specificity,0.7929891,0.9999540,0.0
max absolute_mcc,0.2378871,0.2020328,235.0
max min_per_class_accuracy,0.2541501,0.6144678,224.0
max mean_per_class_accuracy,0.2378871,0.6166729,235.0


Gains/Lift Table: Avg response rate: 24.91 %



...

Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_lift,training_classification_error
,2018-08-27 15:37:18,43.134 sec,0.0,nan,nan,nan,nan,nan
,2018-08-27 15:37:18,43.160 sec,1.0,0.4604021,3.0400903,0.6191406,1.3808835,0.4342314
,2018-08-27 15:37:18,43.184 sec,2.0,0.4622217,3.0649552,0.6139135,1.3075083,0.4753392
,2018-08-27 15:37:18,43.211 sec,3.0,0.4565854,2.4782150,0.6173112,1.2892079,0.4939069
,2018-08-27 15:37:18,43.271 sec,4.0,0.4570880,2.2775610,0.6138851,1.3196701,0.4830467
---,---,---,---,---,---,---,---,---
,2018-08-27 15:37:21,46.479 sec,96.0,0.4234355,0.5388735,0.6564675,1.9516329,0.4727430
,2018-08-27 15:37:21,46.522 sec,97.0,0.4233404,0.5386528,0.6566928,1.9239501,0.4614880
,2018-08-27 15:37:21,46.559 sec,98.0,0.4232879,0.5385259,0.6568276,1.8962674,0.4686000
,2018-08-27 15:37:21,46.590 sec,99.0,0.4232426,0.5384196,0.6569515,1.9377915,0.4531331



See the whole table with table.as_data_frame()
Variable Importances: 


variable,relative_importance,scaled_importance,percentage
user_past_purchases,33983.3007812,1.0,0.3317732
hour,27169.8964844,0.7995073,0.2652551
user_country,15115.2998047,0.4447861,0.1475681
weekday,13479.5039062,0.3966508,0.1315981
email_version,5771.7832031,0.1698417,0.0563489
email_text,3913.1511230,0.1151492,0.0382034
daytime,2114.3168945,0.0622163,0.0206417
day,882.0570068,0.0259556,0.0086114


<bound method ModelBase.coef_norm of >

In [61]:
index = result[result['open_email']==0].index
index = np.random.choice(index, size=31035)
index = np.concatenate((index, np.array(result[result['open_email']==1].index)), axis=0)
second_v = result.loc[index]

In [58]:
linkdf = result[result['open_email']==1]

In [68]:
link_drf = H2ORandomForestEstimator(
        model_id="link",
        ntrees=100,
        #stopping_rounds=2,
        score_each_iteration=True,
        #seed=1000000,
        nfolds=4,
        balance_classes=False,
        histogram_type="AUTO")

link_drf.train(x=feature_names,
         y='open_link',
         training_frame=train_h2o_df)
link_drf.coef_norm

drf Model Build progress: |███████████████████████████████████████████████| 100%
Model Details
H2ORandomForestEstimator :  Distributed Random Forest
Model Key:  link


ModelMetricsBinomial: drf
** Reported on train data. **

MSE: 0.16427074813878284
RMSE: 0.4053032792104979
LogLoss: 0.5117827956247758
Mean Per-Class Error: 0.4269225724261809
AUC: 0.5993954868240665
Gini: 0.19879097364813303
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.1191356696089717: 


,0,1,Error,Rate
0,2129.0,3668.0,0.6327,(3668.0/5797.0)
1,336.0,1108.0,0.2327,(336.0/1444.0)
Total,2465.0,4776.0,0.553,(4004.0/7241.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.1191357,0.3562701,309.0
max f2,0.0526418,0.5611713,369.0
max f0point5,0.2590481,0.2913987,198.0
max accuracy,0.8061989,0.8009943,2.0
max precision,0.8909909,1.0,0.0
max recall,0.0240821,1.0,391.0
max specificity,0.8909909,1.0,0.0
max absolute_mcc,0.2576698,0.1184527,199.0
max min_per_class_accuracy,0.1831458,0.5709850,252.0
max mean_per_class_accuracy,0.1794563,0.5730774,255.0


Gains/Lift Table: Avg response rate: 19.94 %



...

Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_lift,training_classification_error
,2018-08-27 15:38:58,11.221 sec,0.0,nan,nan,nan,nan,nan
,2018-08-27 15:38:58,11.228 sec,1.0,0.4631856,4.0967977,0.5428519,1.0842255,0.7920866
,2018-08-27 15:38:58,11.236 sec,2.0,0.4602068,3.9274436,0.5425397,1.3359207,0.7986270
,2018-08-27 15:38:58,11.243 sec,3.0,0.4466871,2.9158772,0.5497284,1.2120459,0.5903991
,2018-08-27 15:38:58,11.252 sec,4.0,0.4387441,2.5441813,0.5611049,1.2475204,0.5997060
---,---,---,---,---,---,---,---,---
,2018-08-27 15:38:59,12.573 sec,96.0,0.4052533,0.5116907,0.5993973,1.6486169,0.5920453
,2018-08-27 15:38:59,12.586 sec,97.0,0.4052198,0.5116375,0.5994831,1.5799245,0.5522718
,2018-08-27 15:38:59,12.601 sec,98.0,0.4052186,0.5115469,0.5996922,1.5799245,0.6014363
,2018-08-27 15:38:59,12.618 sec,99.0,0.4052388,0.5116223,0.5993095,1.5799245,0.5901119



See the whole table with table.as_data_frame()
Variable Importances: 


variable,relative_importance,scaled_importance,percentage
user_past_purchases,11441.8300781,1.0,0.3514486
hour,9603.8369141,0.8393620,0.2949926
weekday,4622.9946289,0.4040433,0.1420005
user_country,2923.7561035,0.2555322,0.0898064
email_text,1712.9926758,0.1497132,0.0526165
email_version,996.7635498,0.0871157,0.0306167
daytime,809.6002197,0.0707579,0.0248678
day,444.4201050,0.0388417,0.0136509


<bound method ModelBase.coef_norm of >

In [69]:
result.to_csv('second_version.csv')

In [70]:
linkdf.to_csv('linked.csv')